# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('/datasets/data.csv')
print(df['total_income'].head(5))

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
Name: total_income, dtype: float64


In [60]:
# Исправление в ответ на комментарий v_1
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Шаг 2.1 Заполнение пропусков

In [61]:


#проверим корректность заголовков столбцов
print(df.columns)
df=df.rename(columns={"dob_years":"job_years"})
#получим общую информацию о датафрейме
df.info()


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
job_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [62]:
#опишите, какие пропущенные значения вы обнаружили;
df[df['days_employed'].isna()].head(5)

,children,days_employed,job_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [63]:
#проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
len_col1_lack=len(df[df['days_employed'].isna()]) 
len_all=len(df['days_employed'])
print(' какую долю составляют пропущенные значения days_employed: '+str(len_col1_lack/len_all))

 какую долю составляют пропущенные значения days_employed: 0.10099883855981417


In [64]:
#проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
len_col2_lack=len(df[df['total_income'].isna()]) 
print(' какую долю составляют пропущенные значения total_income: '+str(len_col2_lack/len_all))

 какую долю составляют пропущенные значения total_income: 0.10099883855981417


приведите возможные причины появления пропусков в данных;
-Причина - отсутствие официальных данных о трудовом стаже и доходе

объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.
-Таким образом, отсутствующие изначально значения будут меньше всего влиять на достоверность конечного результата(меньше аномалия).



In [65]:
median_income = df['total_income'].median(skipna=True)
print('median_income: '+str(median_income))
df['total_income'] = df['total_income'].fillna(median_income)
print(len(df[df['total_income'].isna()]))

median_income: 145017.93753253992
0


In [66]:
median_days_employed = df['days_employed'].median(skipna=True)
df['days_employed'] = df['days_employed'].fillna(median_days_employed)
print(len(df[df['days_employed'].isna()]))

#получим общую информацию о датафрейме
df.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
job_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [67]:
print(df['children'].unique())
print(df['children'].value_counts())

[ 1  0  3  2 -1  4 20  5]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Также есть отклонения с отрицательными значениями показателя, думаю, это тоже нереалистично. Исправим это заполнив значения медианным значением, рассчитанным из данных, без отклонений.

In [68]:
#'children'
print('children<0: '+str(len(df[df['children']<0])))#отклонения с отрицательными значениями показателя, думаю, это тоже нереалистично

median_children = int(df[df['children']>0]['children'].median(skipna=True))
print('median_children: '+str(median_children)) #медианным значением, рассчитанным из данных, без отклонений.
df.loc[df['children']<0,['children']] = median_children
print('children<0: '+str(len(df[df['children']<0])))

print('children>=20: '+str(len(df[df['children']>=20])))#  количество детей равно 20, у 76 семей, думаю это нереалистичные данные
df.loc[df['children']==20,['children']] = median_children#медианным значением, рассчитанным из данных, без отклонений.
print('children>=20: '+str(len(df[df['children']>=20])))

children<0: 47
median_children: 1
children<0: 0
children>=20: 76
children>=20: 0


In [69]:
#'days_employed'
# вещественный тип данных в столбце  на целочисленный
df['days_employed']=df['days_employed'].astype(int)

#заменяем отрицательные значения 
print('days_employed<0: '+str(len(df[df['days_employed']<0])))

# положительные значения = 365213 это ошибка
df.loc[df['days_employed']==365213,['days_employed']] = 0
print(df[df['days_employed']>0]['days_employed'].tail(5))

df.loc[df['days_employed']<0,['days_employed']] = df.loc[df['days_employed']<0,['days_employed']] * (-1)
print(df[df['days_employed']<0]['days_employed'].tail(5))

print(df['days_employed'].tail(5))

days_employed<0: 18080
21505    338904
21508    386497
21509    362161
21518    373995
21521    343937
Name: days_employed, dtype: int64
Series([], Name: days_employed, dtype: int64)
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, dtype: int64


In [70]:
print('job_years<0: '+str(len(df[df['job_years']<0])))
print('job_years>100: '+str(len(df[df['job_years']>100])))

job_years<0: 0
job_years>100: 0


In [71]:
#переводим к однотипному виду строки с образованием
#видим что данные более полные чем education_id
df.loc[df['education']=='СРЕДНЕЕ','education']= 'среднее'
df.loc[df['education'] == 'Среднее','education']= 'среднее'
df.loc[df['education'] == 'ВЫСШЕЕ','education']= 'высшее'
df.loc[df['education'] == 'Высшее','education']= 'высшее'
df.loc[df['education'] == 'Неоконченное высшее','education']= 'неоконченное высшее'
df.loc[df['education'] == 'НЕОКОНЧЕННОЕ ВЫСШЕЕ','education']= 'неоконченное высшее'
df.loc[df['education'] == 'НАЧАЛЬНОЕ','education']= 'начальное'
df.loc[df['education'] == 'Начальное','education']= 'начальное'
df.loc[df['education'] == 'Ученая степень','education']= 'ученая степень'
df.loc[df['education'] == 'УЧЕНАЯ СТЕПЕНЬ','education']= 'ученая степень'
print(df['education'].value_counts().head(30))

# После удаления дубликатов сделайте следующее:
#     -   поясните, как выбирали метод для поиска и удаления дубликатов в данных;
# - визуально
#     -   приведите возможные причины появления дубликатов.
# - ошибка оператора 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Спасибо за рекомендацию, буду использовать в дальнейшем.

In [72]:
print('education_id<0: '+str(len(df[df['education_id']<0])))
print(df['education_id'].value_counts().head(3)) #double education

education_id<0: 0
1    15233
0     5260
2      744
Name: education_id, dtype: int64


In [73]:
print(df['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [74]:
print(df['family_status_id'].value_counts().head()) #double family_status

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [75]:
print(df['gender'].value_counts())
df.drop(df[df.gender=='XNA'].index, inplace = True)
print(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
F    14236
M     7288
Name: gender, dtype: int64


In [76]:
print(df['income_type'].value_counts())


# После удаления дубликатов сделайте следующее:
#     -   поясните, как выбирали метод для поиска и удаления дубликатов в данных;
# - визуально
#     -   приведите возможные причины появления дубликатов.
# - ошибка оператора 


сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Под влиянием недоказуемых соображений принял неправильное решение. Удалять не надо, исправил.

In [77]:
print('debt<=0: '+str(len(df[df['debt']<=0])))
print('debt>100: '+str(len(df[df['debt']>1000000])))
print(df['debt'].head())
print(df[df['debt']>0]['debt'].head())

debt<=0: 19783
debt>100: 0
0    0
1    0
2    0
3    0
4    0
Name: debt, dtype: int64
14    1
32    1
38    1
55    1
75    1
Name: debt, dtype: int64


In [78]:
print('total_income<0: '+str(len(df[df['total_income']<0])))
print('total_income>1000000: '+str(len(df[df['total_income']>1000000])))
df['total_income']=df['total_income'].astype(int)

total_income<0: 0
total_income>1000000: 25


In [79]:
#Если в данных присутствуют строки-дубликаты, удалите их
print('присутствуют строки-дубликаты: '+str(len(df[df.duplicated()])))
df.drop(df[df.duplicated()].index, inplace = True)
print('присутствуют строки-дубликаты: '+str(len(df[df.duplicated()])))

присутствуют строки-дубликаты: 71
присутствуют строки-дубликаты: 0


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
job_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [81]:
# 5.  Создайте два новых датафрейма со столбцами:
#     -   `education_id` и `education` — в первом;
#     -   `family_status_id` и `family_status` — во втором.

df_ed = df.loc[:,['education_id','education']].copy()
df_fa = df.loc[:,['family_status_id','family_status']].copy()

# Удалите из исходного датафрейма столбцы `education` и `family_status`,
# оставив только их идентификаторы: `education_id` и `family_status_id`.
print(df.info())
df.drop_duplicates(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
job_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
job_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       2145

In [82]:
df_ed =df_ed.drop_duplicates().reset_index(drop=True)
df_fa=df_fa.drop_duplicates().reset_index(drop=True)


In [83]:
df_ed

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [84]:
df_fa

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [85]:
# 6.  На основании диапазонов, указанных ниже, создайте столбец `total_income_category` с категориями:
#     -   0–30000 — `'E'`;
#     -   30001–50000 — `'D'`;
#     -   50001–200000 — `'C'`;
#     -   200001–1000000 — `'B'`;
#     -   1000001 и выше — `'A'`.

def cat_income(total_income):
    if total_income <30000:
        return 'E'
    elif total_income <50000:
        return 'D'
    elif total_income <200000:
        return 'C'
    elif total_income <1000000:
        return 'B'
    elif total_income >1000000:
        return 'A'
    else: return 'X'

print (cat_income(55000))
df['total_income_category']=df['total_income'].apply(cat_income)
df['total_income_category'].head(10)

C


0    B
1    C
2    C
3    B
4    C
5    B
6    B
7    C
8    C
9    C
Name: total_income_category, dtype: object

### Шаг 2.7. Категоризация целей кредита.

In [86]:
# 7.  Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец 
#`purpose_category`, в который войдут следующие категории:
#     -   `'операции с автомобилем'`,
#     -   `'операции с недвижимостью'`,
#     -   `'проведение свадьбы'`,
#     -   `'получение образования'`.

#print(df['purpose'].value_counts())
#варианты с ключевыми словами
# свадьб
# недвижимост жиль
# автомобил
# образован

def cat_purp(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    elif 'жиль' in purpose:
        return 'операции с недвижимостью'
    else: return 'X'

print (cat_purp('автомобилист'))
df['purpose_category']=df['purpose'].apply(cat_purp)
df['purpose_category'].value_counts()

операции с автомобилем


операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Вывод по шагу предобработки данных: В ходе анализа данных были 
    выявлены недостатки, в частности дубликаты, некорректные данные, 
    все они были откорректированы для получения более подходящего материала для дальнейшей обработки.
    

### Ответы на вопросы.

In [87]:
df.pivot_table(index = ['purpose_category'], values =  'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10810,782,0.072340
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


"Проблемные" клиенты это с назначением кредита на операции с автомобилем и получение образования.

In [88]:
df.pivot_table(index = ['family_status_id'], values =  'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
family_status_id,,,
0,12339,931,0.075452
1,4150,388,0.093494
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509


"Проблемные" клиенты это с 1 и 4 типами семей

женат / замужем           0
гражданский брак          1
Не женат / не замужем     4
в разводе                 3
вдовец / вдова            2

In [89]:
df.pivot_table(index = ['total_income_category'], values =  'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5040,356,0.070635
C,16016,1360,0.084915
D,350,21,0.060000
E,22,2,0.090909


"Проблемные" клиенты это с Е и С категорией дохода

In [90]:
 df.pivot_table(index = ['children'], values =  'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14090,1063,0.075444
1,4931,453,0.091868
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


"Проблемные" клиенты это с 1,2 и 4 детьми.

Пересчитал доли по формуле.

## Общий вывод:

Задача:  Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Общий вывод: В проведённом исследовании были использованы данные о платежеспособности различных категорий клиентов. Данные были очищены от аномалий и приведены к виду удобному для получения выводов, а именно - на завершающих таблицах  мы видим доли должников по категориям. Исходя из результатов можно сделать следующий вывод - семейное положение влияет на факт погашения кредита в срок, количество детей тоже влияет, но чуть менее заметно.
